In [2]:
!pip install transformers[torch] datasets evaluate seqeval bitsandbytes accelerate peft trl
!pip install torch
!pip install --upgrade huggingface_hub
!pip install tqdm
!pip install sentencepiece
!pip install protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 121.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 145.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 84.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 140.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 202.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 179.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 22

# LLaVA-v1.6-34B

In [1]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration, BitsAndBytesConfig
from huggingface_hub import login
import torch

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

login(token="FILL IN")
base_model_id = "llava-hf/llava-v1.6-34b-hf" 
processor = LlavaNextProcessor.from_pretrained(base_model_id)
model = LlavaNextForConditionalGeneration.from_pretrained(base_model_id, torch_dtype=torch.float16, load_in_8bit=True, low_cpu_mem_usage=True) #, load_in_8bit=True

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [2]:
model.to("cuda:0")

You shouldn't move a model that is dispatched using accelerate hooks.


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [2]:
from PIL import Image

image_path = './images/image20.png'
image = Image.open(image_path)
# prompt = "USER: <image>\nReturn a list of all the names of companies and people in the email.\nASSISTANT:"
prompt = ('Return a list of all the names of companies and people in the email. If the full name of the person or '+
'company is not given, do not return the name. Also do not return any duplicate names. Each name should be categorized '+
'as either a company or a person. Here is an example of how I would like you to format your output:\n\n'+
"- Name: Elon Musk | Category: person\n"+
"- Name: SpaceX | Category: company\n"+
"- Name: Gwen Shotwell | Category: person\n\n"+
'Return your output in the same format. Do not include any text outside of the format. Obviously replace the names given by '+
'the names found in the email. If you cannot find any names return "NO NAMES FOUND". Return your output below:'
)
full_prompt = f"<|im_start|>system\nAnswer the questions.<|im_end|><|im_start|>user\n{prompt}\n<image><|im_end|><|im_start|>assistant\n"

inputs = processor(full_prompt, image, return_tensors="pt").to("cuda:0")

# autoregressively complete prompt
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=100)

print(processor.decode(output[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<|im_start|> system
Answer the questions.<|im_start|> user
Return a list of all the names of companies and people in the email. If the full name of the person or company is not given, do not return the name. Also do not return any duplicate names. Each name should be categorized as either a company or a person. Here is an example of how I would like you to format your output:

- Name: Elon Musk | Category: person
- Name: SpaceX | Category: company
- Name: Gwen Shotwell | Category: person

Return your output in the same format. Do not include any text outside of the format. Obviously replace the names given by the names found in the email. If you cannot find any names return "NO NAMES FOUND". Return your output below:
<|im_start|> assistant
- Name: Marylita Coyne | Category: person
- Name: Jim | Category


In [ ]:
# 4 bit

# 8 bit
- Name: Maryletticoyne | Category: person
- Name: General Catalyst | Category: company
- Name: MultitOn | Category: company
- Name: Agnetic | Category: company
- Name: IBM | Category: company
- Name: Microsoft | Category: company

# No quant
- Name: Maryletta Coyne | Category: person
- Name: General Catalyst | Category: company
- Name: MultitOn | Category: company
- Name: Agnetic | Category: company
- Name: IBM | Category: company
- Name: Microsoft | Category: company